<img src="viz/Rlogo.png" alt="Drawing" style="width: 200px;"/>

# R Grunnkurs på Dapla
# DEL 1: Introduksjon

#### Beregninger

In [ ]:
2 + 2
5 - 2
25 * 4
5 ^ 2

#### Et lite tall

In [ ]:
3E-2

#### Et stort tall

In [ ]:
3E7

#### log av et tall (natural)

In [ ]:
log(10)

#### exponential tall (e)

In [ ]:
exp(2)

#### en string


In [ ]:
"Hello, world!"

#### lim ting sammen til en string


In [ ]:
paste("Hello", "Susie")
paste("Hello", "Susie", sep = ", ")
paste("Susie er", 39, "år gammel")

#### Plukk ut noen character i en string

In [ ]:
substr("Susie liker kake", 1, 2)
substr("Susie liker kake", 13, 16)

### Objekter

In [ ]:
grunnlagt <- 1669
grunnlagt

In [ ]:
sted <- "Kongsvinger Festning"
sted

#### Objekter kan overskrives

In [ ]:
grunnlagt <- grunnlagt + 1
grunnlagt

#### Objekttype

In [ ]:
str(sted)
str(grunnlagt)

## Vektorer
#### Eksempel på en numerisk vector 

In [ ]:
aar <- c(2018, 2019, 2020)
aar

#### Eksempel på en kategoriske vektor

In [ ]:
flere_sted <- c("Oslo","Kongsvinger", "Rogaland")
flere_sted

#### Hvor mange er i vektoren

In [ ]:
length(flere_sted)

## Logiske setninger

In [ ]:
grunnlagt == 1668
grunnlagt == 1669
aar == 2018

#### Test for motsetninger

In [ ]:
2 != 3

### Test for innhold

In [ ]:
2018 %in% aar
2015 %in% aar

ifelse(sted == "Oslo", "0301", "3401")

# DEL 2: Data
## Lage data (lagret i minne)
Vi kan lage et vanlig datasett ved funksjonen `data.frame()`. Funkjsonen kombinerer flere vektorer av samme lengde til et objekt. Funkjsonen `head()` vises de første 6 linjer. Hvis det er vektorer lagre i minne fra før kan disse kombineres til et datasett eller kan vektorenen spesifiseres. 

In [ ]:
fylkedata <- data.frame(fylke = c("Viken",  "Innlandet"), innbyggere = c(1241000, 371000))
head(fylkedata)

For en oppsummering av datasettet kan vi bruke funkjsonen `summary()`.

In [ ]:
summary(fylkedata)

## Lese inn data fra datalageret (Dapla)

For å lese inn data fra datalageret må funksjonen <code>spark.read.path</code> benyttes. Den ivaretar informasjonssikkerheten knyttet til hvem som har tilgang til å se hvilke data. 

Funksjonen er automatisk tilgjengelig i __SparkR__ kernel.

#### Finne datasett
<code>spark.read.path</code> krever en sti (eng: path) til hvor dataene ligger. For å finne denne stien, så kan vi bruke <code>spark.read.path</code> til å lete:

In [ ]:
df_datasett <- spark.read.path('/felles/*')
head(df_datasett)

#### Lese inn datasettet 'bnp_og_innbyggerantall'

In [ ]:
bnp <- spark.read.path('/felles/veiledning/pyspark/eksempler/bnp_og_innbyggerantall')
head(bnp)

## Skrive data til datalagre (dapla)
Før du lagre data fra prod. sone til dapla bør du lese å gå gjennom SSBs prosedyre for dette.  
[SSBs prosedyre for overføring til Dapla](https://statistics-norway.atlassian.net/wiki/spaces/MS/pages/1648853195/Prosedyre+for+overf+re+data+til+Dapla+fra+prodsonen)

Prosedyren inkluderer verdering av [sikkerhet, tilgangsstyring og verdiverdering](https://statistics-norway.atlassian.net/wiki/spaces/MS/pages/1586204629/Sikkerhet+tilgangstyring+og+verdivurdering). 

Etter verdering er det mulig å lagre data direkte fra en notebook. For å lagre data til datalagre må vi først lagre datasett på et format som passer til datalagringssytemet. Fra R må data være en SparkDataFrame. Vi gjøre dette endringen ved funksjonen `as.DataFrame()`. 

Her er et eksempel av endring i format av datasett fylkedata som vi laget tidligere.

In [ ]:
spark_df <- as.DataFrame(fylkedata)
str(spark_df)

Og deretter kan vi lagre til skyen ved bruk av funksjonen `spark.write.path()`. Vi spesifisere data som skal lagres og hvor, med navn. 

Parameter **valuation** spesifisere beskyttelsesbehov. Det kan ta verdiene:
- "Sensitive" = Data som er sensitiv
- "Shielded" = Skjermet
- "Internal" = Data for intern bruk
- "Open" = Data som er åpen 

Parameter **state** sier noen om hvor i produksjonsprosess data er og kvalitet på det. 
- "Other"
- "Raw" = rådata
- "Input" = inndata
- "Processed" = data som er klargjort
- "Output" = utdata
- "Product"
- "Temp"

For oppdatering av *valuation* og *state* parameterne ser [kode på github](https://github.com/statisticsnorway/dapla-catalog-protobuf/blob/master/src/main/proto/dataset.proto).

In [ ]:
spark.write.path(spark_df, "/felles/veiledning/r/eksempler/fylke", valuation = "INTERNAL", state = "INPUT")

## Les inn data fra Statbank
Data kan leses inn fra Statbank direkte. For dette bruker vi funkjsonen `ApiData()` i tilleggspakke **PxWebApiData**. For å hente data trenger du å vite tabell nummer. Funksjonen returneres en liste hvor det andre elementet er datasettet. Dette kan vises ved å spesifisere`[[2]]`. For mer info se på e-læringskurs her: <https://ssbviz.shinyapps.io/ApiData/>

Følgende eksempel viser innlesing av data fra https://www.ssb.no/statbank/table/03013/

In [ ]:
library(PxWebApiData)
kpi <- ApiData(03013)[[2]]
head(kpi)

## Datasett type
Datasett kan være av forskjellige type i R. For å finne ut hva slags type datasett det er bruker vi `str()`. Det mest standard er 'data.frame'. Når vi lese inn fra Dapla Sky-lagring er data type 'SparkDataFrame'. Ved bruk av tidyverse innlesning (for eksempel `read_csv()`) er data type 'tibble'. Vi kan endre datasett type til en vanlig R data frame ved bruk av `as.data.frame()`. 

In [ ]:
str(bnp)

In [ ]:
bnp <- as.data.frame(bnp)
str(bnp)